# Elastic Client

In [ ]:
from ltr.client import ElasticClient
client = ElasticClient()

# Download & Build Index (run once)

If you don't already have the downloaded dependencies; if you don't have TheMovieDB data indexed run this

In [ ]:
from ltr import download
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'
download([corpus, judgments], dest='data/');

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

## Features for movie titles

We'll be searching movie titles (think searching for a specific movie on Netflix). And we have a set of judgments around the appropriatte movie to return. IE search for "Star Wars" return good star wars matches, in quality order...

These cover various aspects of the problem (searching title by phrase, title bm25 score, release date, etc). We'll use this to explore and analyze a simple model

In [ ]:
# Clear TMDB's LTR
client.reset_ltr(index='tmdb')

In [ ]:
client.reset_ltr(index='tmdb')

config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
            { #1
                "name": "title_has_phrase",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            { #2
                "name": "title_has_terms",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            { #3
                "name": "title_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"title": "{{keywords}}"}
                }
            },
            { #4
                "name": "overview_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"overview": "{{keywords}}"}
                }
            },
            { #5
                "name": "overview_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {"overview": "{{keywords}}"}
                }
            },
            { #6
                "name": "title_fuzzy",
                "params": ["keywords"],
                "template": {
                    "match": {"title": 
                                {"query": "{{keywords}}",
                                 "fuzziness": "AUTO"}}
                }
            },
             { #7
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            }
            
            
            ]
    }}


client.create_featureset(index='tmdb', name='title', ftr_config=config)

## Training Set Generation

Log out features for each of the above queries out to a training set file

In [ ]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/title_judgments.txt', 
                                        trainingOutFile='data/title_judgments_train.txt', 
                                        featureSet='title')

In [ ]:
# stranger things <- 'bad query' -> 1
#  Grades 0-4
# 1
# 2
# 2
# 0
# 0
# --> 2,2,1,0,0 <-- Perfect NDCG DCG / iDCG -> 1.0
# iDCG is pretty bad
# Ideal DCG 5
#
# rocky <- good query -> 1
# 4
# 4
# 4
# 3
# Ideal DCG 25
#
#
#
# 4      <= iDCG
# 4
# 4
# 4  

# rambo
# rocky
# star wars
# K fold cross validation
# K=3 fold cross validation
#
# Fold 1
# - train on two examples - rambo, rocky, target DCG@4
# - test on the other one - star wars, how did we do DCG@4 = 10
# Fold 2
# - train on two examples - rambo, star wars, target DCG@4
# - test on the other one - rocky, how did we do DCG@4 = 12
# Fold 3
# - train on two examples - rocky, star wars, target DCG@4
# - test on the other one - rambo, how did we do DCG@4 = 14
#
# Training 20 <-- overfit!
# AVG all the tests, 12
#
# Training 13 <-- not too bad! Our model can generalize!
# AVG all the tests, 12
#
#
# Full train on all the data -> model




# P@N <-- precision
# NCDG@N <-- NDCG
# DCG@N <-- DCG
# ERR@N <-- ERR
# MAP@N <-- Mean Avg Precision
download(['http://es-learn-to-rank.labs.o19s.com/RankyMcRankFace.jar'], dest='data/');

# LambdaMART / Gradient Boosting / MART
#
# (Ensemble for features 1..7)
#  (Decision Trees) 6,3,2       
#  (Decision Trees) 6,3,2
#  (Decision Trees) 6,3,2
#  (Decision Trees) 6,3,2
#  (Decision Trees) 6,3,2
#  (Decision Trees) 4,5,7
#  (Decision Trees) 4,5,7
#  (Decision Trees) 4,5,7
#  (Decision Trees) 4,5,7
#  (Decision Trees) 7,1,2

#
# Random Forests (bag=2, frate=0.6)
#
# 5->3
# (Ensemble for 40% of features 1..7) 1,3,5,7,
#  (Decision Trees) 5,3,1
#  (Decision Trees) 5,3,1
#  (Decision Trees) 5,3,1
#  (Decision Trees) 5,3,1
#  (Decision Trees) 5,7,1
#  (Decision Trees) ...
#  (Decision Trees)
#  (Decision Trees)
#  (Decision Trees)
#  (Decision Trees)
#
# 6->2
# (Ensemble for features 40% of features 1..7) 2,3,5,6
#  (Decision Trees) 6,2,3
#  (Decision Trees) 6,2,3
#  (Decision Trees) 6,2,3
#  (Decision Trees) 5,6,2
#  (Decision Trees) ..
#  (Decision Trees)
#  (Decision Trees)
#  (Decision Trees)
#  (Decision Trees)
#  (Decision Trees)

# Feature 3 reduced error 12250.938430747574
# Feature 6 reduced error 12167.279980776944
# Feature 7 reduced error 11184.936996662022
# Feature 4 reduced error 7342.442106757739
# Feature 5 reduced error 5252.231534271401
# Feature 1 reduced error 2586.7181503571337
# Feature 2 reduced error 126.93687012857046

#
#-- FEATURE IMPACTS
# Feature 7 reduced error 8.795923250386467E8
# Feature 5 reduced error 7.232922151350935E8
# Feature 1 reduced error 3.301986939497073E8
# Feature 4 reduced error 3.1184694617029667E8
# Feature 3 reduced error 2.5817983556409964E8
# Feature 2 reduced error 0.0
# Feature 6 reduced error 0.0


!java -jar data/RankyMcRankFace.jar -ranker 6 -bag 10 -frate 0.6 -tree 10 \
 -metric2t DCG@4 -train data/title_judgments_train.txt -save data/model.txt 

## Feature Search: which features work best?

What combination of these features work best? Train a model with every combination, and use k-fold cross valudation (see `kcv=15` below). The combination with the best NDCG is output

In [ ]:
from ltr.train import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               trainingInFile='data/title_judgments_train.txt',
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               features=[1,2,3,4,5,6,7],
                                               featureSet='title')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

## Compare to model w/ all features

Compare the features output above (something like...)

```
Impact of each feature on the model
3 - 17728.651362443747
4 - 10986.88505569509
7 - 7504.573843344015
1 - 1580.321225819952
```

to one trained with the full model. Notice how features have different impacts. This is due to feature dependency

In [ ]:
from ltr import train
trainLog  = train(client,
                  trainingInFile='data/title_judgments_train.txt',
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  features=[3,4,7,1],
                  index='tmdb',
                  featureSet='title',
                  modelName='title')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

## Bias towards fewer features

By adding a 'cost', to feature search, we add a multiplier that punishes models with more features slightly. This results in a tiny bias towards simpler models all things being equal. As we'd prefer one that doesn't need to execute more features

In [ ]:
from ltr.train import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               trainingInFile='data/title_judgments_train.txt',
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               featureCost=0.02,# 1.0-cost ^ num_features
                                               features=[1,2,3,4,5,6,7],
                                               featureSet='title')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

# Evaluating the Model

It's interesting to see what features our model makes use of, but we need guidance on adding additional features to the model. We know our model is an ensemble of decision trees. Wouldn't it be cool if we could trace where documents end up on that decision tree?

Specifically, we care about problems. Or what we will call affectionately *whoopsies*. 

As a 'whoopsie' example, consider the query "Rambo". if a '0' document like 'First Daughter' ranked the same or higher than a '4' document ("Rambo")., that's a problem. It's also an opportunity for improvement. We'd want to isolate that, see if it's indicative of a broader trend, and thus worth adding a feature for.

Let's see a concrete example

In [ ]:
from ltr.client import ElasticClient
from ltr.MART_model import eval_model
from ltr.judgments import judgments_from_file, judgments_by_qid

features, _ = client.feature_set(index='tmdb', name='title')

with open('data/title_judgments_train.txt') as f:
    judgmentDict = judgments_by_qid(judgments_from_file(f))


rambo=judgmentDict[1]
model = eval_model(modelName='title',
                   features=features,
                   judgments=rambo)

print()
print("## Evaluating graded docs for search keywords '%s'" % rambo[0].keywords)
print()
print(model)

## Examining our evaluation for whoopsies

Let's looks at one tree in our ensemble, te see how it was evaluated.

```
if title_bm25 > 10.664251:
  if title_phrase > 0.0:
    if title_bm25 > 13.815164:
      if release_year > 2000.0:
        <= 0.1215(0/0/)
      else:
        <= 0.1240(0/0/)
    else:
      if title_bm25 > 10.667803:
        if overview_bm25 > 0.0:
          <= 0.1194(0/0/)
        else:
          <= 0.1161(1/0/)
      else:
        <= 0.1264(0/0/)
  else:
    <= 0.0800(0/0/)
else:
  if title_phrase > 0.0:
    if title_bm25 > 8.115499:
      if title_bm25 > 8.217656:
        <= 0.1097(2/1/qid:40:2(12180)-3(140607))
      else:
        <= 0.1559(0/0/)
    else:
      <= -0.0021(2/1/qid:40:2(1895)-3(330459))
  else:
    <= -0.1093(25/1/qid:40:0(85783)-3(1892))
```

You'll notice here this tree is represented by a series of if statements, where the feature's name is used. This is handy as it lets us take apart the structure of the tree.

You'll also notice the leaf nodes starting with 

```
<=
```

These leaf nodes have a floating point value, corresponding to the relevance score that documents ending up here will have. Each leaf also has three items in paranthesis, such as `(2/1/qid:40:2(1895)-3(330459))`. This is a report summarizing the result of evaluating the tree on the provided judgment list. Indicating:


```



   +--- 2 Documents evaluated to this leaf node                   +-- max grade doc eval'd to this leaf
   |                                                              |
   | +----- 1 'whoopsie' occured                                  |  +-- corresp. doc id of max doc
   | |                                                            |  |
   | |   +--- details on each whoopsie ----------- qid:40:2(1985)-3(330459)
   | |   |                                              | |  |
  (2/1/qid:40:2(1895)-3(330459))                        | |  |
                                                        | |  + doc id of min graded doc
                                                        | |
                                                        | + min grade of docs eval'd to this leaf
                                                        |
                                                        + query id of whoopsie from judgments
```


Looking at Star Wars, our biggest issues in this tree are with the bottom-most leaf. Here

```
if title_bm25 > 10.664251:
  ...
else:
  if title_phrase > 0.0:
    ...
  else:
    <= -0.1093(25/1/qid:40:0(85783)-3(1892))
```


Document 85783 (a '0') and doc 1892 are given the same grade.

In [ ]:
### Doc in index... (notice nothing mentions 'star wars')

client.get_doc(index='tmdb', doc_id=1368)

In [ ]:
client.get_doc(index='tmdb', doc_id=1368)

## Add a feature: collection name

We have an intuition about our data, there is a field for the movies "collection name". See it here below:

In [ ]:
from ltr.helpers.movies import get_movie
get_movie(1368)

We'll add collection name, and reindex.

In [ ]:
from ltr import download
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

from ltr.client import ElasticClient
client = ElasticClient()

def add_char_names(src_movie, base_doc):
    if 'belongs_to_collection' in src_movie and src_movie['belongs_to_collection'] is not None:
        if 'name' in src_movie['belongs_to_collection']:
            base_doc['collection_name'] = src_movie['belongs_to_collection']['name']
    if 'cast' in src_movie and src_movie['cast'] is not None:
        characters = []
        for cast_entry in src_movie['cast']:
            characters.append(cast_entry['character'])
        base_doc['secondary_characters'] = characters[10:]
        base_doc['prom_characters'] = characters[:10]
    return base_doc


movies=indexable_movies(movies='data/tmdb.json', enrich=add_char_names)
rebuild(client, index='tmdb', doc_src=movies)

Confirm it's in our doc now...

In [ ]:
client.get_doc(index='tmdb', doc_id=1368)

Add it to the features, and retrain....

In [ ]:
config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [ 
            {  #1
                "name": "title_phrase",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            { #2
                "name": "title",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            { #3
                "name": "title_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"title": "{{keywords}}"}
                }
            },
            { #4
                "name": "overview_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"overview": "{{keywords}}"}
                }
            },
            { #5
                "name": "overview_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {"overview": "{{keywords}}"}
                }
            },
            { #6
                "name": "title_fuzzy",
                "params": ["keywords"],
                "template": {
                    "match": {"title": 
                                {"query": "{{keywords}}",
                                 "fuzziness": "AUTO"}}
                }
            },
             { #7
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            },
            { #8
                "name": "secondary_char_name_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"secondary_characters": 
                                {"query": "{{keywords}}"}}
                }
            },
            { #9
                "name": "prom_char_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"prom_characters": 
                                {"query": "{{keywords}}"}}
                }
            },
            { # 10
                "name": "coll_name_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"collection_name": 
                                {"query": "{{keywords}}"}}
                }
            },
            { # 11
                "name": "coll_name_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {"collection_name": 
                                {"query": "{{keywords}}"}}
                }
            }
            
            
            ] #qid:1 red high heels -> keywords, query_understanding_color, item_type
    }}

client.create_featureset(index='tmdb', name='title2', ftr_config=config)

from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/title_judgments.txt', 
                                        trainingOutFile='data/title2_judgments_train.txt', 
                                        featureSet='title2')

In [ ]:
from ltr.train import kcv
res  = kcv(client,
            trainingInFile='data/title2_judgments_train.txt',
            metric2t='NDCG@10',
            leafs=20,
            trees=20,
            ranker=8, # Use a "Random Forests Model"
            frate=0.5,
            bag=10, # Number of ensembles in the forest bag=1, 1 LambdaMART model with random features chosen
            index='tmdb',
            kcv=5,
            features=[1,2,3,4,5,6,7,8,9,10,11],
            featureSet='title2',
            modelName='title2')

print()
print("Impact of each feature on the model")
for ftrId, impact in res.trainingLogs[0].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
    
print("Test NDCG@10 %s" % res.kcvTestAvg)

In [ ]:
from ltr.train import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               trainingInFile='data/title2_judgments_train.txt',
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               features=[1,2,3,4,5,6,7,8,9],
                                               featureSet='title2')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

In [ ]:
from ltr import train
trainLog  = train(client,
                  trainingInFile='data/title2_judgments_train.txt',
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  index='tmdb',
                  features=[1,4,6,7,8],
                  featureSet='title2',
                  modelName='title2')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

In [ ]:
from ltr.train import kcv
res  = kcv(client,
            trainingInFile='data/title2_judgments_train.txt',
            metric2t='NDCG@10',
            leafs=4,
            trees=100,
            ranker=8, # Use a "Random Forests Model"
            frate=0.5,
            bag=100, # Number of ensembles in the forest bag=1, 1 LambdaMART model with random features chosen
            index='tmdb',
            kcv=5,
            features=[1,2,3,4,5,6,7,8,9],
            featureSet='title2',
            modelName='title_rf')

print()
print("Impact of each feature on the model")
for ftrId, impact in res.trainingLogs[0].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
    
print("Test NDCG@10 %s" % res.kcvTestAvg)

In [ ]:
from ltr import search
search(client, "basketball cartoon aliens", modelName='title2')

In [ ]:
client.reset_ltr(index='tmdb')

config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
            {
                "name": "title_phrase",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
 
            {
                "name": "overview_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"overview": "{{keywords}}"}
                }
            },
   
            {
                "name": "title_fuzzy",
                "params": ["keywords"],
                "template": {
                    "match": {"title": 
                                {"query": "{{keywords}}",
                                 "fuzziness": "AUTO"}}
                }
            },
             {
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            },
            {
                "name": "coll_name_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"collection_name": 
                                {"query": "{{keywords}}"}}
                }
            },
            {
                "name": "title_fuzzy_2",
                "params": ["keywords"],
                "template": {
                       "match": {"title.as_str": 
                                {"query": "{{keywords}}",
                                 "fuzziness": "AUTO"}}
                }
            }
            ]
    }}


client.create_featureset(index='tmdb', name='title2', ftr_config=config)

from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/title_judgments.txt', 
                                        trainingOutFile='data/title2_judgments_train.txt', 
                                        featureSet='title2')

In [ ]:
client.get_doc(index='tmdb', doc_id=1892)

In [ ]:
from ltr.train import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               trainingInFile='data/title2_judgments_train.txt',
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               features=[1,2,3,4,5],
                                               featureSet='title2')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

In [ ]:
from ltr import train
trainLog  = train(client,
                  trainingInFile='data/title2_judgments_train.txt',
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  index='tmdb',
                  featureSet='title2',
                  modelName='title2')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

In [ ]:
from ltr.client import ElasticClient
from ltr.MART_model import eval_model
from ltr.judgments import judgments_from_file, judgments_by_qid

features, _ = client.feature_set(index='tmdb', name='title2')

with open('data/title_judgments_train.txt') as f:
    judgmentDict = judgments_by_qid(judgments_from_file(f))

for qid, judgments in judgmentDict.items():

    model = eval_model(modelName='title2',
                           features=features,
                           judgments=judgments)

    print()
    print("## Evaluating graded docs for search keywords '%s'" % judgments[0].keywords)
    print()
    print(model)

In [ ]:
client.get_doc(index='tmdb', doc_id=860)